In [1]:
import pandas as pd
import os
import numpy as np
from dataclasses import dataclass
import random
import time



np.random.seed(14)

In [2]:
@dataclass
class Particle:
    x: float
    y: float
    theta: float
    weight: float 
        


@dataclass
class LandMark:
    x: float
    y: float
    index: int

In [3]:
gt_data = pd.read_csv('data/gt_data.txt', names=['X','Y','Orientation'], sep=' ')
map_data = pd.read_csv('data/map_data.txt', names=['X','Y','# landmark'])
control_data = pd.read_csv('data/control_data.txt', names=['velocity','Yaw rate'], sep=' ')

#observation = pd.read_csv('data/observation/observations_000001.txt', names = ['X cord','Y cord'], sep=' ')


result = [(x,y, landmark) for x,y,landmark in zip(map_data['X'],map_data['Y'], map_data['# landmark'])]
landarkList=[]
for res in result:
    l = LandMark(res[0],res[1],res[2])
    landarkList.append(l)

    

a = os.listdir("data/SensorDataFiles")
a.sort()
observation=[]
for i in range(len(a)):
    fileName = 'data/SensorDataFiles/'+a[i]
    observationTmp = pd.read_csv(fileName, names = ['X cord','Y cord'], sep=' ')
    observation.append(observationTmp)

In [4]:
def calculateDistance(landmark1, landmark2):
    a =  np.sqrt((landmark1.x - landmark2.x)**2 + (landmark1.y - landmark2.y)**2)
    return a

def findClosestLandmark(map_landmarks, singleObs):
    #todo: write code:
    # take the single obs and convert to global coordinates
    
    closest_landmark = map_landmarks[0]
    # check which landmark is the closest
    for i in range(len(map_landmarks)):
        if calculateDistance(singleObs, map_landmarks[i]) < calculateDistance(singleObs, closest_landmark):
            closest_landmark = map_landmarks[i]
    return closest_landmark
    
    
    
        
def getError(gt_data, bestParticle):
    error1 = np.abs(gt_data[0] - bestParticle.x)
    error2 = np.abs(gt_data[1] - bestParticle.y)
    error3 = np.abs(gt_data[2] - bestParticle.theta)
    if(error3>2*np.pi):
        error3 = 2*np.pi - error3
    return (error1, error2, error3)

def findObservationProbability(closest_landmark,map_coordinates, sigmaX, sigmaY):
    
    mew_x = closest_landmark.x
    mew_y = closest_landmark.y
    
    x = map_coordinates.x;
    y = map_coordinates.y;
    
    frac =  (1/ (2 * np.pi * sigmaX * sigmaY ))
    weight1 = (x-mew_x)**2/((sigmaX)**2)  +(y-mew_y)**2/(sigmaY**2)    
    ans = frac * np.exp(-1*weight1)
    return ans



def mapObservationToMapCoordinates(observation, particle):
    x = observation.x
    y = observation.y

    xt = particle.x
    yt = particle.y
    theta = particle.theta

    MapX = x * np.cos(theta) - y * np.sin(theta) + xt
    MapY = x * np.sin(theta) + y * np.cos(theta) + yt
    return MapX, MapY

def mapObservationsToMapCordinatesList(observations, particle):
    
    convertedObservations=[]
    i=0
    for obs in observations.iterrows():
        singleObs = LandMark(obs[1][0],obs[1][1],1)
        mapX, mapY = mapObservationToMapCoordinates(singleObs, particle)
        tmpLandmark = LandMark(x=mapX, y=mapY, index=i)
        i+=1
        convertedObservations.append(tmpLandmark)
    return convertedObservations



In [5]:
class ParticleFilter:
    particles = []
    def __init__(self, intialX, initialY, std, numOfParticles):
        self.number_of_particles = numOfParticles
        for i in range(self.number_of_particles):
           # tmpParticle = Particle(intialX,initialY,0,1)
            x = random.gauss(intialX, std)
            y = random.gauss(initialY, std)
            theta = random.uniform(0, 2*np.pi)
            tmpParticle = Particle(x,y , theta,1)
            self.particles.append(tmpParticle)



    def moveParticles(self, velocity, yaw_rate, delta_t=0.1):
        
        for i in range(self.number_of_particles):
            if(yaw_rate!=0):
                theta = self.particles[i].theta
                newTheta = theta + delta_t*yaw_rate;
                newX =  self.particles[i].x +(velocity/yaw_rate)*(np.sin(newTheta)-np.sin(theta));
                newY =  self.particles[i].y +(velocity/yaw_rate)*(np.cos(theta)-np.cos(newTheta));

                #todo Add noise!!
                self.particles[i].x = newX +random.gauss(0, 0.3)
                self.particles[i].y = newY +random.gauss(0, 0.3)
                self.particles[i].theta = newTheta +random.gauss(0, 0.01)
            else:
                print("ZERO!!!")
                
                
    
    def UpdateWeight(self, observations):
        
        for i in range(self.number_of_particles):
            map_obser = mapObservationsToMapCordinatesList(observations,self.particles[i])
            prob = 1.00000000000000000000000000000
            #print(self.particles[i])
            for obs in map_obser:
                    #print(obs)
                    land = findClosestLandmark(landarkList,obs)
                    #print(land)
                    prob *= findObservationProbability(land, self.particles[i], abs(land.x - obs.x),abs(land.y-obs.y))
            #print(prob)
            self.particles[i].weight = prob
        
    def getBestParticle(self):
        best_particle =  max(self.particles, key= lambda particle: particle.weight)
        return best_particle    

    def getBestParticleOut(self):
        x=0
        y=0
        theta=0
        for i in range(self.number_of_particles):
            x+= self.particles[i].x
            y+= self.particles[i].y
            theta+= self.particles[i].theta
        x=x/self.number_of_particles
        y=y/self.number_of_particles
        theta=theta/self.number_of_particles
        best_particle =  Particle(x,y,theta, weight=1)
        return best_particle        

    def PrintWeights(self):
        for i in range(self.number_of_particles):
            print("Weight:",self.particles[i].weight, self.particles[i].x,self.particles[i].y)

    def Resample(self):
        N = self.number_of_particles
        sumWeights = 0.0
        weights= []
        
        for i in range(N):
            sumWeights+=self.particles[i].weight
        
        for i in range(N):
            self.particles[i].weight = round(self.particles[i].weight / sumWeights,20)
            weights.append(self.particles[i].weight)
        particles_new = []
        
        for i in range(N):
            particles_new.append(np.random.choice(self.particles,1,weights))
        self.particles.clear()
        print(particles_new[0][0])
        for i in range(N):
            self.particles.append(particles_new[i][0])

In [6]:
sigmaY = 0.3

magicNumberOfParticles = 200


start = time.time()

def main():
    particleFilter = ParticleFilter(0 ,0 ,0.3, numOfParticles=magicNumberOfParticles)
    #articleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=magicNumberOfParticles)
   
    for i in range(len(observation)):
        #prediction
        if(i!=0):
            velocity = control_data.iloc[i-1][0]
            yaw_rate = control_data.iloc[i-1][1]
            particleFilter.moveParticles(velocity, yaw_rate)
        
        
        a = observation[i].copy()
        particleFilter.UpdateWeight(a)
        particleFilter.Resample()
        bestP = particleFilter.getBestParticle()
        print(bestP)
        error = getError(gt_data.iloc[i], bestP)
        print(i,error)
    end = time.time()
    print(end - start)
        
main()

Particle(x=-0.600792493092522, y=-0.2103883901426121, theta=2.432990398365697, weight=0.0)
Particle(x=-0.15945173812358668, y=-0.1971299884927625, theta=1.010770489282447, weight=0.001975354310174102)
0 (6.437951738123587, 2.1569299884927626, 1.010770489282447)
Particle(x=-0.5477240110038517, y=-1.1828102143429884, theta=4.340245159629139, weight=0.0)
Particle(x=-0.4732493922581926, y=-0.5978754939539758, theta=3.0722843020341553, weight=7.342913524046118e+107)
1 (7.136449392258193, 2.6803754939539757, 2.7629143020341553)
Particle(x=-1.7249336294196773, y=-0.8389749529550885, theta=3.360278209428708, weight=0.0)
Particle(x=-1.2679604335415735, y=0.3077117160675089, theta=3.2282630147601457, weight=9.423807095183716e+144)
2 (8.323360433541573, 1.8998882839324909, 2.9197030147601457)


<ipython-input-5-74df6385d941>:78: RuntimeWarning: overflow encountered in double_scalars
  self.particles[i].weight = round(self.particles[i].weight / sumWeights,20)


Particle(x=2.3835734457483113, y=-3.5060808490159907, theta=5.275920882352442, weight=0.0)
Particle(x=2.4539477915536465, y=-5.051308475044319, theta=5.493793717159066, weight=inf)
3 (5.002152208446354, 7.387108475044319, 5.1846737171590656)
Particle(x=-4.102849562999138, y=9.197961715993461, theta=1.9389644808166695, weight=0.0)
Particle(x=4.0538231394289275, y=-6.938423191030155, theta=5.506157595202777, weight=inf)
4 (3.8117768605710722, 9.405523191030154, 5.195937595202777)
Particle(x=-4.3162175293923415, y=14.615946592930545, theta=1.9323444690119578, weight=0.0)
Particle(x=12.149671959079118, y=-4.162241102110051, theta=6.057598673316079, weight=inf)
5 (3.8644719590791183, 6.763341102110051, 5.7474786733160785)
Particle(x=-4.3341146902197245, y=0.7214296625536426, theta=3.2151730822391524, weight=0.0)
Particle(x=19.504042642240467, y=-5.988890739701614, theta=6.032540907380524, weight=inf)
6 (10.789642642240468, 8.726390739701614, 5.723320907380524)
Particle(x=6.886181395576991, 

<ipython-input-5-74df6385d941>:78: RuntimeWarning: overflow encountered in multiply
  self.particles[i].weight = round(self.particles[i].weight / sumWeights,20)


Particle(x=-15.631737732336493, y=0.1394356877675602, theta=2.9899052138538713, weight=inf)
Particle(x=-15.631737732336493, y=0.1394356877675602, theta=2.9899052138538713, weight=inf)
8 (25.234837732336494, 2.8793643122324397, 2.6838552138538714)
Particle(x=7.66578715957352, y=-5.772999116265018, theta=5.79257988868557, weight=0.0)
Particle(x=13.099719976166126, y=-6.4507513367921385, theta=6.077889625820362, weight=inf)
9 (3.0367199761661254, 9.615951336792138, 5.769989625820362)
Particle(x=17.115434295669004, y=-6.645477569607554, theta=6.081262124644991, weight=0.0)
Particle(x=7.32926466078184, y=8.743425549128569, theta=1.1327549054316057, weight=inf)
10 (3.20273533921816, 5.4298255491285685, 0.8242249054316058)
Particle(x=19.632803585169196, y=-26.973919022125706, theta=5.281247066947979, weight=0.0)
Particle(x=-19.741109697274947, y=9.349800616073285, theta=2.764279681932883, weight=inf)
11 (30.749109697274946, 5.886000616073285, 2.457599681932883)
Particle(x=24.907543600674114, 

KeyboardInterrupt: 

In [12]:
from math import *
import random
import sys
print(sys.version)

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

class robot:
	def __init__(self):
		self.x = random.random()*world_size
		self.y = random.random()*world_size
		self.orientation = random.random()*2.0*pi
		self.forward_noise = 0.0
		self.turn_noise    = 0.0
		self.sense_noise   = 0.0


	def set(self,new_x,new_y,new_orientation):
		if new_x < 0 or new_x >= world_size:
			raise ValueError ('x coordinate out of bound')

		if new_y< 0 or new_y>=world_size:
			raise valueError ('Y coordinate out of bound')
		if new_orientation < 0 or new_orientation >= 2*pi:
			raise ValueError ('Orientation must be in [0..2pi]')

		self.x = float(new_x)
		self.y = float(new_y)
		self.orientation = float(new_orientation)

	def set_noise(self,new_f_noise, new_t_noise, new_s_noise):

		self.forward_noise = float(new_f_noise);
		self.turn_noise = float(new_t_noise);
		self.sense_noise = float(new_s_noise);


	def sense(self):
		Z = []
		for i in range(len(landmarks)):
			dist = sqrt((self.x - landmarks[i][0])**2 + (self.y- landmarks[i][1]) **2)
			dist += random.gauss(0.0,self.sense_noise)
			Z.append(dist)
		return Z

	def move(self,turn,forward):
		if forward < 0:
			raise ValueError ('Robot cant move backwards')

		orientation = self.orientation + float(turn) + random.gauss(0.0,self.turn_noise)
		orientation %= 2*pi

		dist = float(forward) + random.gauss(0.0, self.forward_noise)
		x = self.x + (cos(orientation)*dist)
		y = self.y + (sin(orientation)*dist)
		x %= world_size
		y %= world_size


		res = robot()
		res.set(x,y,orientation)
		res.set_noise(self.forward_noise,self.turn_noise, self.sense_noise)
		return res

	def Gaussian(self,mu,sigma,x):

		return exp(-((mu-x)**2)/(sigma**2)/2.0)/sqrt(2.0*pi*(sigma**2))

	def measurement_prob(self,measurement):

		prob = 1.0;
		for i in range(len(landmarks)):
			dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
			prob *= self.Gaussian(dist, self.sense_noise, measurement[i])

		return prob

	def __repr__(self):
		return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))	

def eval(r,p):
	sum = 0.0;
	for i in range(len(p)):
		dx  = (p[i].x-r.x+(world_size/2.0))% world_size - (world_size/2.0)
		dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
		err = sqrt(dx * dx + dy * dy)
		sum += err
	return sum / float(len(p))	


####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

myrobot = robot()
# enter code here
forward_noise = 5.0
turn_noise = 0.1
sense_noise = 5.0
myrobot.set_noise(5.0,0.1,5.0)

myrobot.set(30.0, 50.0, pi/2)
myrobot = myrobot.move(-pi/2, 15.0)
print (myrobot.sense())
myrobot = myrobot.move(-pi/2, 10.0)
print (myrobot.sense())


# Now we want to create particles,
# p[i] = robot(). In this assignment, write
# code that will assign 1000 such particles
# to a list.
#
# Your program should print out the length
# of your list (don't cheat by making an
# arbitrary list of 1000 elements!)
#
# Don't modify the code below. Please enter
# your code at the bottom.

N = 1000
p = []

myrobot = robot()
myrobot = myrobot.move(0.1,5.0)
Z = myrobot.sense()

for i in range(N):
    x = robot()
    x.set_noise(0.05,0.05,5.0)
    p.append(x)


p2 = []
for i in range(N):
    p2.append(p[i].move(0.1,5.0))
p = p2

w = []

for i in range(N):
    w.append(p[i].measurement_prob(Z))
    
print(w)

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
[41.47389152671661, 45.233372872233424, 38.03294243045048, 51.07333897590731]
[35.723688599063976, 51.46178573985218, 54.959243122798256, 43.45178053585916]
[3.203669969485549e-58, 4.0083216746211755e-51, 8.619708953465147e-73, 2.5519042002436516e-10, 2.340557096199196e-32, 5.308407709243835e-26, 1.810217219873818e-36, 6.125066675264895e-24, 7.88225263015889e-52, 6.897119541426487e-72, 1.408182640024874e-38, 8.560404568929394e-28, 3.1146561971550714e-63, 1.38347492208842e-65, 4.622895209845789e-97, 1.080880692736238e-19, 3.5758237751934206e-40, 3.920375203902085e-11, 5.576591359487336e-22, 6.027998239603872e-07, 1.4773110215968272e-83, 1.1537693386271062e-27, 2.7403367169215428e-71, 1.1128768503859046e-17, 1.1478998589814786e-29, 1.5153918015058782e-11, 4.780401115643463e-08, 2.925357054211338e-75, 2.53940753996519e-17, 1.6959047947358308e-94, 7.44421134414437e-93, 7.4609931640251775e-28, 1.0876285127923816e-17, 2.15645